In [ ]:
# default_exp core

# fastproaudio core

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
assert say_hello("Jeremy")=="Hello Jeremy!"
